In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms
from torch.utils.data import DataLoader

class VGGBlock(nn.Module):
    def __init__(self,in_channels,out_channels,num_convs):
        super().__init__()

        layers = []
        for i in range(num_convs):
            layers.append(
                nn.Conv2d(
                    in_channels if i == 0 else out_channels,
                    out_channels,
                    kernel_size = 3,
                    padding = 1
                )
            )
            layers.append(
                nn.BatchNorm2d(num_features = out_channels)
            )
            layers.append(
                nn.ReLU(inplace = True)
            )
        layers.append(
            nn.MaxPool2d(
                kernel_size = 2, stride = 2
            )
        ) # shape reduces to half

        self.block = nn.Sequential(*layers)

    def forward(self,x):
        return self.block(x)

In [16]:
class VGGNet_CIFAR(nn.Module):
    def __init__(self,num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            VGGBlock(in_channels = 3, out_channels = 64, num_convs =2), # Block 1 with 2 convs & 1 maxpool  Shape (64, 16, 16)
            VGGBlock(in_channels=64 , out_channels=128 , num_convs = 3), # Block2  with 3 convs & 1 maxpool  Shape (128, 8, 8)
            VGGBlock(in_channels=128 , out_channels=256 , num_convs = 3), # Block3  with 3 convs & 1 maxpool Shape (256, 4, 4)
            VGGBlock(in_channels=256 , out_channels=512 , num_convs = 3), # Block4  with 3 convs & 1 maxpool Shape (512, 2, 2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 2 * 2,4096),
            nn.ReLU(inplace = True),
            nn.Dropout(p = 0.5),

            nn.Linear(4096,4096),
            nn.ReLU(inplace = True),
            nn.Dropout(p = 0.5),

            nn.Linear(4096, num_classes)
        )

    def forward(self,x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [17]:
def train_CIFAR(architecture_class,optimizer_name = 'Adam',epochs = 10):
    # Data loading and transformation

    transform_train = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding=4),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=(0.4914, 0.4822, 0.4465),
            std=(0.2470, 0.2435, 0.2616)
        )
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
            mean=(0.4914, 0.4822, 0.4465),
            std=(0.2470, 0.2435, 0.2616)
        )
    ])

    train_dataset = datasets.CIFAR10(root="./data-CIFAR",download = True, train=True, transform=transform_train)
    test_dataset  = datasets.CIFAR10(root="./data-CIFAR",download = True, train=False, transform=transform_test)

    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
    test_loader  = DataLoader(test_dataset, batch_size=128, shuffle=False)

    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Optimizer Part

    model = architecture_class().to(device)
    criterion = nn.CrossEntropyLoss()
    if optimizer_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=1e-3)
    elif optimizer_name == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    else:
        print("Unknown optimizer! Defaulting to Adam.")
        optimizer = optim.Adam(model.parameters(), lr=1e-3)

    # ------- training loop --------

    epochs = epochs

    for epoch in range(epochs):
        model.train()
        correct = 0
        total = 0
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = outputs.max(1)
            total += labels.size(0)
            correct += preds.eq(labels).sum().item()

        print(f"Epoch [{epoch+1}/{epochs}] "
            f"Loss: {running_loss/len(train_loader):.4f} "
            f"Train Acc: {100*correct/total:.2f}%")
    
    # Test Evaluation Part

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = outputs.max(1)
            total += labels.size(0)
            correct += preds.eq(labels).sum().item()

    print(f"Test Accuracy: {100*correct/total:.2f}%")



In [19]:
train_CIFAR(architecture_class=VGGNet_CIFAR,optimizer_name='Adam',epochs = 30)

Epoch [1/30] Loss: 2.1165 Train Acc: 17.38%
Epoch [2/30] Loss: 1.8515 Train Acc: 24.83%
Epoch [3/30] Loss: 1.6581 Train Acc: 33.84%
Epoch [4/30] Loss: 1.5088 Train Acc: 41.72%
Epoch [5/30] Loss: 1.3403 Train Acc: 49.91%
Epoch [6/30] Loss: 1.1162 Train Acc: 61.05%
Epoch [7/30] Loss: 0.9366 Train Acc: 67.61%
Epoch [8/30] Loss: 0.8313 Train Acc: 71.60%
Epoch [9/30] Loss: 0.7426 Train Acc: 75.11%
Epoch [10/30] Loss: 0.6684 Train Acc: 77.94%
Epoch [11/30] Loss: 0.6052 Train Acc: 80.02%
Epoch [12/30] Loss: 0.5733 Train Acc: 81.16%
Epoch [13/30] Loss: 0.5282 Train Acc: 82.76%
Epoch [14/30] Loss: 0.4961 Train Acc: 83.91%
Epoch [15/30] Loss: 0.4613 Train Acc: 85.02%
Epoch [16/30] Loss: 0.4379 Train Acc: 85.75%
Epoch [17/30] Loss: 0.4155 Train Acc: 86.38%
Epoch [18/30] Loss: 0.3905 Train Acc: 87.27%
Epoch [19/30] Loss: 0.3709 Train Acc: 87.88%
Epoch [20/30] Loss: 0.3529 Train Acc: 88.61%
Epoch [21/30] Loss: 0.3287 Train Acc: 89.39%
Epoch [22/30] Loss: 0.3205 Train Acc: 89.60%
Epoch [23/30] Loss: